In [1]:
# (a)
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

# Defining s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
key_file = 'framingham.csv'

bucket_object = bucket.Object(key_file)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading csv
heart = pd.read_csv(file_content_stream)

# (b)
heart = heart.dropna()
heart.head(1)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0


In [2]:
heart = heart.dropna()

In [9]:
# Defining predictors and target variable
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

Y.value_counts() / Y.shape[0]

0    0.847648
1    0.152352
Name: TenYearCHD, dtype: float64

Notice that the range of 0 is almost 85%. This means that when we are splitting the data, we need to take the propotion of 0s into account by specifying stratify = y

In [20]:
# Splitting the dataset
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, stratify = Y, test_size = 0.2)

In [21]:
Y_train.value_counts() / Y_train.shape[0]

0    0.847811
1    0.152189
Name: TenYearCHD, dtype: float64

In [22]:
# Transforming data to 0-1 scale
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [23]:
## Logistic regression
md1 = LogisticRegression().fit(X_train, Y_train)

# Predicting on test
pred1 = md1.predict_proba(X_test)[:,1]
pred1 = np.where(pred1 < 0.1, 0, 1)

# Computing the recall
recall_score(Y_test, pred1)

0.8839285714285714

In [24]:
# Random Forest
md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

# Predicting on test
pred2 = md2.predict_proba(X_test)[:,1]
pred2 = np.where(pred2 < 0.1, 0, 1)

# Computing the recall
recall_score(Y_test, pred2)

0.9017857142857143

#### Based on recall, I would use Randeom Forest Classifier, which is from part 6 to predict TenYearCHD.